<a href="https://colab.research.google.com/github/gamecicn/TF2_Study/blob/KaggleMinst/T2E1_Kaggle_MINST_TransferL_v1_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.optimizers import Adam


from tensorflow import keras
from tensorflow.data import Dataset
from tensorflow.keras import layers
from tensorflow.keras import Model

In [2]:
# Download Data
csv_train_file = '/content/drive/MyDrive/DS_data/MINSTHW/train.csv'
csv_test_file = '/content/drive/MyDrive/DS_data/MINSTHW/test.csv'

df_data = pd.read_csv(csv_train_file)
df_test = pd.read_csv(csv_test_file)

df_data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Preprcessing

In [3]:
# Split train data to train and valid dataset
df_data_copy = df_data.copy()

df_train  = df_data_copy.sample(frac=0.99, random_state=0)
df_valid = df_data_copy.drop(df_train.index)

print("lenght of train is {}".format(len(df_train)))
print("lenght of valid is {}".format(len(df_valid)))

train_labels = df_train.pop('label')
valid_labels = df_valid.pop('label')

df_train.head()

arr_train = df_train.to_numpy().reshape(-1, 28 ,28, 1)
arr_valid = df_valid.to_numpy().reshape(-1, 28 ,28, 1)


lenght of train is 41580
lenght of valid is 420


In [4]:
arr_train[0].shape

(28, 28, 1)

# Preprocessing

# Resize

In [5]:
# Processs Map 
INPUT_IMG_SIZE = (32, 32)

def resize_img(x):
  return tf.image.resize(x, INPUT_IMG_SIZE).numpy()

train_data = np.array(list(map(resize_img, arr_train)))
train_valid = np.array(list(map(resize_img, arr_valid)))

In [8]:
train_data.shape

(41580, 32, 32, 1)

## Single Channel to Multi Channel




In [6]:
TARGET_CHANNEL = 3

def dupe_channel(x):
  return np.tile(x, (1, 1, TARGET_CHANNEL))

train_data = np.array(list(map(dupe_channel, train_data)))
train_valid = np.array(list(map(dupe_channel, train_valid)))



# Build the model

### Prepare Image

In [58]:
### 导入模型
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.applications.resnet import ResNet50


IMG_SHAPE = (32, 32, 3)

# Pre-layer 
input_layer = tf.keras.layers.Input(shape=IMG_SHAPE)

resnet_preprocess_layer = preprocess_input(input_layer)
base_model = ResNet50(input_shape = (32, 32, 3), include_top = False) 
base_model.trainable = False 
base_model = base_model(resnet_preprocess_layer, training=False)

# base_model = tf.keras.layers.Conv2D(32, (2, 2), activation='relu')(input_layer)
# base_model = tf.keras.layers.MaxPooling2D((2, 2))(base_model)
# base_model = tf.keras.layers.Dropout(0.2)(base_model)

# 加入全连接层
x = layers.Flatten()(base_model)
x = layers.Dense(64, activation='relu')(x) 
x = layers.Dense(10)(x)

# 构建模型序列
model = Model(input_layer, x)


In [41]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(28, (2, 2), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10)
])



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 27, 27, 28)        140       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 13, 13, 28)        0         
_________________________________________________________________
dropout_23 (Dropout)         (None, 13, 13, 28)        0         
_________________________________________________________________
flatten_15 (Flatten)         (None, 4732)              0         
_________________________________________________________________
dense_28 (Dense)             (None, 64)                302912    
_________________________________________________________________
dense_29 (Dense)             (None, 10)                650       
Total params: 303,702
Trainable params: 303,702
Non-trainable params: 0
________________________________________________

In [59]:
model.summary()

Model: "model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
tf.__operators__.getitem_8 ( (None, 32, 32, 3)         0         
_________________________________________________________________
tf.nn.bias_add_8 (TFOpLambda (None, 32, 32, 3)         0         
_________________________________________________________________
resnet50 (Functional)        (None, 1, 1, 2048)        23587712  
_________________________________________________________________
flatten_21 (Flatten)         (None, 2048)              0         
_________________________________________________________________
dense_40 (Dense)             (None, 128)               262272    
_________________________________________________________________
dropout_33 (Dropout)         (None, 128)               0  

In [62]:
# Compile Model
model.compile(optimizer=keras.optimizers.Adam(), 
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics = ['acc'])

# model.summary()

history = model.fit(x = train_data,
                    y = train_labels,
                    batch_size = 64,
                    validation_data = (train_valid, valid_labels),
                    epochs=10)

Epoch 1/10
650/650 [==============================] - 11s 13ms/step - loss: 0.1019 - acc: 0.9645 - val_loss: 0.1559 - val_acc: 0.9524
Epoch 2/10
650/650 [==============================] - 8s 12ms/step - loss: 0.0985 - acc: 0.9649 - val_loss: 0.1120 - val_acc: 0.9667
Epoch 3/10
650/650 [==============================] - 8s 12ms/step - loss: 0.0955 - acc: 0.9670 - val_loss: 0.1472 - val_acc: 0.9500
Epoch 4/10
650/650 [==============================] - 8s 12ms/step - loss: 0.0888 - acc: 0.9696 - val_loss: 0.1319 - val_acc: 0.9595
Epoch 5/10
650/650 [==============================] - 8s 12ms/step - loss: 0.0837 - acc: 0.9715 - val_loss: 0.1097 - val_acc: 0.9738
Epoch 6/10
650/650 [==============================] - 8s 12ms/step - loss: 0.0797 - acc: 0.9720 - val_loss: 0.1397 - val_acc: 0.9524
Epoch 7/10
650/650 [==============================] - 8s 12ms/step - loss: 0.0854 - acc: 0.9700 - val_loss: 0.1337 - val_acc: 0.9643
Epoch 8/10
650/650 [==============================] - 8s 12ms/step -